In [186]:
import numpy as np
import pandas as pd
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping

# Load and prepare data

In [187]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [188]:
rawData = pd.read_csv("../input/train.csv").values
rawSubmission = pd.read_csv("../input/test.csv").values

In [190]:
#suffle dataset
np.random.shuffle(rawData)

trainSamples = rawData.shape[0] # amount of samples in training set
testSamples = rawSubmission.shape[0] # amount of samples in test set
imgSize = 28 # image size (both width and height)
colorChannels = 1 # images are grayscale

# reshape and normalize
X = rawData[:, 1:].astype("float32").reshape( (trainSamples, imgSize, imgSize, colorChannels) ) / 255.0
testX = rawSubmission.astype("float32").reshape( (testSamples, imgSize, imgSize, colorChannels) ) / 255.0

Y = np_utils.to_categorical( rawData[:,0].astype("int") )

numClasses = Y.shape[1]

# Build the model

In [191]:
# we will use 3 conv layers with 32 filters with size 5x5

model = Sequential()

model.add(Convolution2D(32, 5, 5, input_shape=(imgSize, imgSize, colorChannels), 
                        border_mode='same', activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.2))

model.add(Convolution2D(32, 5, 5, activation='relu', border_mode='same', W_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Convolution2D(32, 5, 5, activation='relu', border_mode='same', W_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(numClasses, activation='softmax'))

In [192]:
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# stop training if val_loss stop decreasing after 2 epoch
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_66 (Convolution2D) (None, 28, 28, 32)    832         convolution2d_input_25[0][0]     
____________________________________________________________________________________________________
dropout_66 (Dropout)             (None, 28, 28, 32)    0           convolution2d_66[0][0]           
____________________________________________________________________________________________________
convolution2d_67 (Convolution2D) (None, 28, 28, 32)    25632       dropout_66[0][0]                 
____________________________________________________________________________________________________
maxpooling2d_42 (MaxPooling2D)   (None, 14, 14, 32)    0           convolution2d_67[0][0]           
___________________________________________________________________________________________

# Fit model

In [123]:
# validation_split = 0.25 - split oyr dataset: train on 75% of samples and validate on 25%
model.fit(X, Y, validation_split = 0.25, nb_epoch=epochs, batch_size=32, callbacks=[early_stopping])

Train on 31500 samples, validate on 10500 samples
Epoch 1/25
31500/31500 [==============================] - 389s - loss: 0.3120 - acc: 0.8974 - val_loss: 0.0703 - val_acc: 0.9771
Epoch 2/25
31500/31500 [==============================] - 384s - loss: 0.0756 - acc: 0.9760 - val_loss: 0.0510 - val_acc: 0.9835
Epoch 3/25
31500/31500 [==============================] - 382s - loss: 0.0550 - acc: 0.9829 - val_loss: 0.0442 - val_acc: 0.9862
Epoch 4/25
31500/31500 [==============================] - 370s - loss: 0.0428 - acc: 0.9862 - val_loss: 0.0398 - val_acc: 0.9890
Epoch 5/25
31500/31500 [==============================] - 355s - loss: 0.0353 - acc: 0.9889 - val_loss: 0.0384 - val_acc: 0.9890
Epoch 6/25
31500/31500 [==============================] - 374s - loss: 0.0285 - acc: 0.9908 - val_loss: 0.0377 - val_acc: 0.9891
Epoch 7/25
31500/31500 [==============================] - 379s - loss: 0.0253 - acc: 0.9920 - val_loss: 0.0369 - val_acc: 0.9896
Epoch 8/25
31500/31500 [=======================

### Optionaly save model to .h5 file

In [124]:
# model.save('convnet_9910.h5')

# Prepare submission

In [137]:
predicted = model.predict_classes(testX)

28000/28000 [==============================] - 110s   


In [185]:
submission = np.column_stack( (np.arange(1, predicted.shape[0]+1), predicted) )
np.savetxt('convnet_v1.csv', submission, delimiter=',', header = 'ImageId,Label', fmt="%d", comments='')